<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Стоп-слова" data-toc-modified-id="Стоп-слова-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Стоп-слова</a></span></li><li><span><a href="#Токенизация" data-toc-modified-id="Токенизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Токенизация</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разделение-на-признаки-и-векторизация" data-toc-modified-id="Разделение-на-признаки-и-векторизация-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделение на признаки и векторизация</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Решающий-лес" data-toc-modified-id="Решающий-лес-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Решающий лес</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.sample(10)

,text,toxic
67507,"FYI \n\nHi Durova,\n\nWasn't sure if you'd not...",0
32477,The hutu/twa are considered the ancient parent...,0
63171,"""\n\nLet me answer these in order...\n\nI am n...",0
100706,hello cow head,1
26931,"""\n\nBus ServiceDestinationFrequencyJourney Ti...",0
122202,"It's back again. I think there's a bug, Ill tr...",0
4468,"""Just face the truth: you can't escape it!==\n...",1
152644,"""\n\n Please do not remove content from pages ...",0
86396,Removal of the Dred Scott Comparison - Complet...,0
26230,One question to the admins: If you intend to l...,0


In [3]:
data.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Наши данные несбалансированы, положительных ответов всего 10 %

In [4]:
short_data=data.sample(frac=0.5).reset_index(drop='True')
short_data

,text,toxic
0,There's an interesting article about artists' ...,0
1,Image making \n\nHey. I just wanted to ask you...,0
2,"As Marge Simpson says, there's no shame in bei...",0
3,Volkswagen \n\nI wasn't vandalising you little...,0
4,IP editor Welcome to Wikipedia.If you want to ...,0
...,...,...
79781,"""\n\n BI \n\nMick, I guess I'm going to be blo...",0
79782,Permanent Block? \n\nI admit to doing wrong an...,0
79783,ref 118 \n\nThe abstract for this ref (don't h...,0
79784,I have made the changes detailed in the steps ...,0


In [5]:
short_data.describe()

,toxic
count,79786.000000
mean,0.100343
std,0.300459
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Баланс данных сохранён.

In [6]:
def clear_text(text):
    clear_text=re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text=clear_text.split()
    return ' '.join(clear_text)

data['text']=data['text'].apply(clear_text)
data['text']

In [7]:
short_data['text']=short_data['text'].apply(clear_text)
short_data['text']

0        There s an interesting article about artists s...
1        Image making Hey I just wanted to ask you what...
2        As Marge Simpson says there s no shame in bein...
3        Volkswagen I wasn t vandalising you little sh ...
4        IP editor Welcome to Wikipedia If you want to ...
                               ...                        
79781    BI Mick I guess I m going to be blocked for su...
79782    Permanent Block I admit to doing wrong and I d...
79783    ref The abstract for this ref don t have the p...
79784    I have made the changes detailed in the steps ...
79785    For length and style we might look at an artic...
Name: text, Length: 79786, dtype: object

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [9]:
short_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79786 entries, 0 to 79785
Data columns (total 2 columns):
text     79786 non-null object
toxic    79786 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


stop = stopwords.words('english')
data['text_without_stopwords'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(data['text_without_stopwords'])

### Стоп-слова

In [10]:
stop = stopwords.words('english')
short_data['text_without_stopwords'] = short_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(short_data['text_without_stopwords'])

0        There interesting article artists statements talk
1        Image making Hey I wanted ask type software us...
2        As Marge Simpson says shame pariah Particularl...
3                       Volkswagen I vandalising little sh
4        IP editor Welcome Wikipedia If want test pleas...
                               ...                        
79781    BI Mick I guess I going blocked supporting Ne ...
79782    Permanent Block I admit wrong I doubt permanen...
79783    ref The abstract ref paper seems talk solution...
79784    I made changes detailed steps I also reduced m...
79785    For length style might look article like Paul ...
Name: text_without_stopwords, Length: 79786, dtype: object


tt = TweetTokenizer()
data['tokenised_text'] = data['text_without_stopwords'].apply(tt.tokenize)
print(data['tokenised_text'])

### Токенизация

In [11]:
tt = TweetTokenizer()
short_data['tokenised_text'] = short_data['text_without_stopwords'].apply(tt.tokenize)
print(short_data['tokenised_text'])

0        [There, interesting, article, artists, stateme...
1        [Image, making, Hey, I, wanted, ask, type, sof...
2        [As, Marge, Simpson, says, shame, pariah, Part...
3                 [Volkswagen, I, vandalising, little, sh]
4        [IP, editor, Welcome, Wikipedia, If, want, tes...
                               ...                        
79781    [BI, Mick, I, guess, I, going, blocked, suppor...
79782    [Permanent, Block, I, admit, wrong, I, doubt, ...
79783    [ref, The, abstract, ref, paper, seems, talk, ...
79784    [I, made, changes, detailed, steps, I, also, r...
79785    [For, length, style, might, look, article, lik...
Name: tokenised_text, Length: 79786, dtype: object


lmtzr = WordNetLemmatizer()
data['lemm_text'] = data['tokenised_text'].apply(
                    lambda lst:[lmtzr.lemmatize(word) for word in lst])
print(data['lemm_text'])

### Лемматизация

In [12]:
lmtzr = WordNetLemmatizer()
short_data['lemm_text'] = short_data['tokenised_text'].apply(
                    lambda lst:[lmtzr.lemmatize(word) for word in lst])
print(short_data['lemm_text'])

0        [There, interesting, article, artist, statemen...
1        [Image, making, Hey, I, wanted, ask, type, sof...
2        [As, Marge, Simpson, say, shame, pariah, Parti...
3                 [Volkswagen, I, vandalising, little, sh]
4        [IP, editor, Welcome, Wikipedia, If, want, tes...
                               ...                        
79781    [BI, Mick, I, guess, I, going, blocked, suppor...
79782    [Permanent, Block, I, admit, wrong, I, doubt, ...
79783    [ref, The, abstract, ref, paper, seems, talk, ...
79784    [I, made, change, detailed, step, I, also, red...
79785    [For, length, style, might, look, article, lik...
Name: lemm_text, Length: 79786, dtype: object


data['lemm_text']=data['lemm_text'].apply(' '.join)
data['lemm_text']

In [13]:
short_data['lemm_text']=short_data['lemm_text'].apply(' '.join)
short_data['lemm_text']

0          There interesting article artist statement talk
1        Image making Hey I wanted ask type software us...
2        As Marge Simpson say shame pariah Particularly...
3                       Volkswagen I vandalising little sh
4        IP editor Welcome Wikipedia If want test pleas...
                               ...                        
79781    BI Mick I guess I going blocked supporting Ne ...
79782    Permanent Block I admit wrong I doubt permanen...
79783    ref The abstract ref paper seems talk solution...
79784    I made change detailed step I also reduced mil...
79785    For length style might look article like Paul ...
Name: lemm_text, Length: 79786, dtype: object

Обработали текст, удалили стоп-слова и провели лемматизацию.

## Обучение

### Разделение на признаки и векторизация

train, test = train_test_split(data, test_size=0.25, random_state=12345)

In [14]:
s_train, s_test = train_test_split(short_data, test_size=0.25, random_state=12345)

In [15]:
stopwords = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

corpus_train = train['lemm_text'].values.astype('U')
corpus_test = test['lemm_text'].values.astype('U')

In [16]:
corpus_s_train = s_train['lemm_text'].values.astype('U')
corpus_s_test = s_test['lemm_text'].values.astype('U')

tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

In [17]:
tf_idf_s_train = count_tf_idf.fit_transform(corpus_s_train)
tf_idf_s_test = count_tf_idf.transform(corpus_s_test)

tf_idf_train.shape

tf_idf_test.shape

In [18]:
tf_idf_s_train.shape

(59839, 91872)

In [19]:
tf_idf_s_test.shape

(19947, 91872)

train_f=tf_idf_train
train_t=train['toxic']
test_f=tf_idf_test
test_t=test['toxic']

In [20]:
s_train_f=tf_idf_s_train
s_train_t=s_train['toxic']
s_test_f=tf_idf_s_test
s_test_t=s_test['toxic']

def pred(model):
    model.fit(train_f,train_t)
    prediction=model.predict(test_f)
    score=f1_score(prediction,test_t)
    print(score)

In [21]:
def short_pred(model):
    model.fit(s_train_f,s_train_t)
    prediction=model.predict(s_test_f)
    score=f1_score(prediction,s_test_t)
    print(score)

### Логистическая регрессия

In [22]:
model=LogisticRegression(class_weight='balanced',random_state=12345,C=10)

In [23]:
short_pred(model)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7665071770334927


### Решающее дерево

In [24]:
model=DecisionTreeClassifier(class_weight='balanced',random_state=12345)

In [25]:
%%time
short_pred(model)

0.6253621573434366
CPU times: user 1min 4s, sys: 0 ns, total: 1min 4s
Wall time: 1min 4s


### Решающий лес

In [26]:
model=RandomForestClassifier(class_weight='balanced',random_state=12345)

In [27]:
%%time
short_pred(model)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5611263736263736
CPU times: user 23.6 s, sys: 0 ns, total: 23.6 s
Wall time: 23.6 s


### LGBMClassifier

In [28]:
model=LGBMClassifier(iterations=500,class_weight='balanced',random_state=12345)

In [29]:
%%time
model.fit(s_train_f,s_train_t)

CPU times: user 11min 22s, sys: 7.08 s, total: 11min 29s
Wall time: 11min 33s


LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', iterations=500,
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective=None, random_state=12345,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [30]:
prediction=model.predict(s_test_f)
f1_score(prediction,s_test_t)

0.7209145931405515

### CatBoostClassifier

In [31]:
model=CatBoostClassifier(iterations=500,loss_function='Logloss',class_weights=(1,10),random_state=12345)

In [32]:
%%time
short_pred(model)

Learning rate set to 0.098907
0:	learn: 0.6491928	total: 3.52s	remaining: 29m 14s
1:	learn: 0.6213731	total: 6.48s	remaining: 26m 53s
2:	learn: 0.6038083	total: 9.5s	remaining: 26m 13s
3:	learn: 0.5857083	total: 12.4s	remaining: 25m 37s
4:	learn: 0.5777110	total: 15.4s	remaining: 25m 23s
5:	learn: 0.5664322	total: 18.3s	remaining: 25m 5s
6:	learn: 0.5577543	total: 21.2s	remaining: 24m 51s
7:	learn: 0.5496519	total: 24.1s	remaining: 24m 39s
8:	learn: 0.5425533	total: 27s	remaining: 24m 31s
9:	learn: 0.5344278	total: 29.9s	remaining: 24m 23s
10:	learn: 0.5285804	total: 32.8s	remaining: 24m 17s
11:	learn: 0.5220457	total: 35.7s	remaining: 24m 10s
12:	learn: 0.5158769	total: 38.7s	remaining: 24m 8s
13:	learn: 0.5123445	total: 41.6s	remaining: 24m 2s
14:	learn: 0.5068839	total: 44.5s	remaining: 23m 57s
15:	learn: 0.5036671	total: 47.4s	remaining: 23m 52s
16:	learn: 0.4981727	total: 50.1s	remaining: 23m 42s
17:	learn: 0.4949431	total: 53s	remaining: 23m 38s
18:	learn: 0.4909674	total: 55.9s	

## Выводы

С теми ресурсами, которые мне были доступны, я смог подобрать только Логистическую регрессию с балансом весов. Она же и на полной выборке показала результат 0.748. 

Пробововал ограничивать количество признаков, но на результат это не сильно влияло. Решил урезать исходные данные, чтобы ядро постоянно не падало.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны